In [ ]:
# =============================================================================
# PHÂN TÍCH NGƯỜI CHƠI TENNIS - SỬ DỤNG PlayerAnalysisService
# =============================================================================
# Chạy cell này để phân tích video và nhận kết quả JSON
# Hình ảnh/video trả về dạng link: outputs/{request_id}/filename.png

import sys
import os
import json
import uuid

sys.path.insert(0, os.path.abspath('.'))

from src.core.player_analysis_service import PlayerAnalysisService

# ====== CẤU HÌNH ======
VIDEO_PATH = "1765199807.mp4"

# Tọa độ 12 điểm sân đã xác định
# Điểm 3 (index 2) và điểm 9 (index 8) là đường lưới
COURT_POINTS = [
    (361, 139),
    (481, 132),
    (560, 130),   # Điểm 3 - bắt đầu lưới
    (664, 131),
    (981, 153),
    (887, 338),
    (714, 641),
    (372, 457),
    (288, 408),   # Điểm 9 - kết thúc lưới
    (244, 372),
    (169, 324),
    (270, 224)
]

# Tạo request_id và output folder
REQUEST_ID = uuid.uuid4().hex[:12]
OUTPUT_DIR = f"outputs/{REQUEST_ID}"

print("=" * 70)
print("🎾 PHÂN TÍCH NGƯỜI CHƠI TENNIS")
print("=" * 70)
print(f"📹 Video: {VIDEO_PATH}")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"🆔 Request ID: {REQUEST_ID}")

In [ ]:
# =============================================================================
# CHẠY PHÂN TÍCH VÀ TRẢ VỀ JSON
# =============================================================================

# Khởi tạo service
service = PlayerAnalysisService(
    ball_model_path="models/ball_best.pt",
    person_model_path="yolo11m.pt",
    pose_model_path="yolo11m-pose.pt"
)

# Chạy phân tích
result = service.analyze(
    video_path=VIDEO_PATH,
    court_points=COURT_POINTS,
    output_folder=OUTPUT_DIR,
    net_start_idx=2,      # Điểm 3
    net_end_idx=8,        # Điểm 9
    ball_conf=0.3,
    person_conf=0.3,
    angle_threshold=50,
    intersection_threshold=50,
    base_url=f"outputs/{REQUEST_ID}"  # Link dạng: outputs/abc123/file.png
)

# Thêm request_id
result["request_id"] = REQUEST_ID

print("\n" + "=" * 70)
print("✅ PHÂN TÍCH HOÀN TẤT!")
print("=" * 70)

In [ ]:
# =============================================================================
# HIỂN THỊ KẾT QUẢ JSON
# =============================================================================

print("=" * 70)
print("📋 KẾT QUẢ JSON")
print("=" * 70)

# In JSON đẹp
print(json.dumps(result, indent=2, ensure_ascii=False, default=str))

In [ ]:
# =============================================================================
# LƯU KẾT QUẢ JSON RA FILE
# =============================================================================

json_path = f"{OUTPUT_DIR}/analysis_results.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2, default=str)

print(f"✅ Đã lưu JSON tại: {json_path}")

# Hiển thị tóm tắt
print("\n" + "=" * 70)
print("📊 TÓM TẮT KẾT QUẢ")
print("=" * 70)
print(f"🆔 Request ID: {result['request_id']}")
print(f"📹 Video: {result['video_info']['duration_seconds']}s, {result['video_info']['fps']} FPS")
print(f"👥 Số người chơi: {result['summary']['total_players']}")
print(f"🎾 Tổng cú đánh: {result['summary']['total_hits']}")
print(f"   ✅ Trong sân: {result['summary']['total_in_court']}")
print(f"   ❌ Ngoài sân: {result['summary']['total_out_court']}")
print(f"   🚫 Không qua lưới: {result['summary']['total_not_over_net']}")

print("\n🏆 BẢNG XẾP HẠNG:")
for r in result['rankings']:
    print(f"   #{r['rank']} Player {r['player_id']}: {r['score']:.2f} điểm, {r['total_hits']} cú đánh")

print("\n📁 CÁC FILE ĐÃ TẠO:")
print(f"   - {result['visualizations']['ranking_board']}")
print(f"   - {result['visualizations']['speed_comparison']}")
for pid, files in result['visualizations']['per_player'].items():
    for name, path in files.items():
        print(f"   - {path}")

In [ ]:
# =============================================================================
# HIỂN THỊ HÌNH ẢNH (OPTIONAL - CHẠY TRONG JUPYTER)
# =============================================================================

from IPython.display import Image, display
import glob

print("=" * 70)
print("🖼️ HIỂN THỊ HÌNH ẢNH")
print("=" * 70)

# Hiển thị ranking board
ranking_path = f"{OUTPUT_DIR}/ranking_board.png"
if os.path.exists(ranking_path):
    print("\n🏆 Bảng xếp hạng:")
    display(Image(filename=ranking_path))

# Hiển thị heatmap từng người
heatmap_files = glob.glob(f"{OUTPUT_DIR}/player_*_heatmap.png")
for hf in heatmap_files:
    print(f"\n🗺️ {os.path.basename(hf)}:")
    display(Image(filename=hf))

# Hiển thị biểu đồ tốc độ
speed_path = f"{OUTPUT_DIR}/speed_comparison.png"
if os.path.exists(speed_path):
    print("\n⚡ Biểu đồ tốc độ:")
    display(Image(filename=speed_path))